In [11]:
from dotenv import load_dotenv
load_dotenv()
import os

pinecone_api = os.getenv(key="pinecone")
hf_api=os.getenv(key="hf-api")


In [4]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone,ServerlessSpec
index_name ="hybrid-search-langchain-pinecone"
#initilize pinecone key
pc =Pinecone(api_key=pinecone_api)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec= ServerlessSpec(cloud='aws',region='us-east-1')
    )

In [5]:
index= pc.Index(index_name)
index

In [12]:
#vector embedding and sparse matrix
from langchain_huggingface import HuggingFaceEmbeddings

embeddings =HuggingFaceEmbeddings(model_name ='all-MiniLM-L6-v2')
embeddings

c:\Users\Nikhillappy\projects\nikhil\Hybrid Search\venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nikhillappy\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Nikhillappy\projects\nikhil\Hybrid Search\venv\Lib\site

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
from pinecone_text.sparse import BM25Encoder

bm25 =BM25Encoder().default()
bm25

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nikhillappy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [15]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Nikhillappy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [16]:
sentences =[
    "The quick brown fox jumps over the lazy dog",
    "The five boxing wizards jump quickly at dawn",
    "Pack my box with five dozen liquor jugs",
]
#tfidf 
bm25.fit(sentences)

#store in json
bm25.dump("bm25_values.json")

bm25_encoder =BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 49.07it/s]


In [17]:
retriever =PineconeHybridSearchRetriever(embeddings =embeddings,sparse_encoder =bm25_encoder, index =index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000024ACACEE600>, index=<pinecone.data.index.Index object at 0x0000024A86BDBAA0>)

In [18]:
retriever.add_texts([
    "The quick brown fox jumps over the lazy dog",
    "The five boxing wizards jump quickly at dawn",
    "Pack my box with five dozen liquor jugs",
]
)

100%|██████████| 1/1 [00:04<00:00,  4.67s/it]


In [21]:
retriever.invoke("how many liquor jugs")

[Document(metadata={'score': 0.485265225}, page_content='Pack my box with five dozen liquor jugs'),
 Document(metadata={'score': 0.0384633541}, page_content='The five boxing wizards jump quickly at dawn'),
 Document(metadata={'score': -0.011025277}, page_content='The quick brown fox jumps over the lazy dog')]